In [11]:
import numpy as np
import cv2
import rosbag
from cv_bridge import CvBridge
from ros_numpy.point_cloud2 import pointcloud2_to_xyz_array
import pcl
from tf.transformations import quaternion_matrix, quaternion_from_euler
import cv2
import ros_numpy
import pandas as pd

# Standard plotly imports
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

In [4]:
# Rosbag path
bag_path = '/home/user/rosbags/repeat/blob/2022-01-20-16-35-51.bag'

# Load rosbag
bag = rosbag.Bag(bag_path)

# ROS topics
camera_topic = '/zed2i/zed_node/left/image_rect_color'
lidar_topic = '/velodyne_points'

# Frame to show (is equivalent to time: frame/10 s)
frame = 100

# Transformation from lidar to camera left optical (using output from tf node)
trans = [0.05999999999999983, -0.3898697311173096, -0.33652837214493014]
quat = [-0.4473902184573541, 0.4473902184573541, -0.5475782979891378, -0.5475782979891378]
# (Euler (rpy)):0.000 -78.500 90.000

Some functions

In [5]:
def unpack_bag(bag_path, topic):
    """Extracts all msgs (with time) of a topic from a rosbag"""
    # Load rosbag
    bag = rosbag.Bag(bag_path)

    t_msg = []
    msgs = []
    for topic, msg, t in bag.read_messages(topics=topic):
        msgs.append(msg)
        # Time at which msg was recorded
        t_msg.append(t.to_time())
    return msgs, t_msg


def synchronize_topics(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Synchronizes two topics (even if different rate)."""
    if len(topic1) == len(topic2):
        return topic1, topic2

    # Make sure both topics start around the same time and remove first values of
    # signal which is ahead if this is not the case
    topic1, t_topic1, topic2, t_topic2 = correct_for_time_diff(
        topic1, t_topic1, topic2, t_topic2)

    # Assume first array is bigger
    array_big = t_topic1
    array_small = t_topic2
    is_topic1_bigger = True
    # Swap if not
    if len(t_topic1) < len(t_topic2):
        array_small, array_big = array_big, array_small
        is_topic1_bigger = False
    indices = []

    # For each time of the small array, check which is
    # the closest msg in the big array
    for i, v in enumerate(array_small):
        # Offset big array with a msg from small array
        diff = np.abs(array_big - v)
        # Index where time difference is the smallest
        idx = diff.argmin()
        if diff[idx] < t_thresh:
            indices.append(idx)

    print(
        "t_start_diff: {:.3f} s, t_end_diff: {:.3f} s".format(
            np.abs(array_small[0] - array_big[indices][0]),
            np.abs(array_big[indices][-1] - array_small[len(array_big[indices]) - 1]),
        )
    )

    if is_topic1_bigger:
        topic1_synched = topic1[indices]
        topic2_synched = topic2[: len(topic1_synched)]
        return topic1_synched, topic2_synched
    else:
        topic2_synched = topic2[indices]
        topic1_synched = topic1[: len(topic2_synched)]
        return topic1_synched, topic2_synched


def correct_for_time_diff(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Checks for a time offset of two topics at the beginning and shorts ahead signal"""
    # Assume first topic is behind and convert to numpy array
    topic_behind, topic_ahead = np.asarray(topic1), np.asarray(topic2)
    t_topic_behind, t_topic_ahead = np.asarray(t_topic1), np.asarray(t_topic2)
    is_topic1_behind = True
    # Swap if other way around
    if t_topic2[0] - t_topic1[0] > t_thresh:
        topic_ahead, topic_behind = topic_behind, topic_ahead
        t_topic_ahead, t_topic_behind = t_topic_behind, t_topic_ahead
        is_topic1_behind = False
    # No offset
    elif np.abs(t_topic1[0] - t_topic2[0]) < t_thresh:
        return tuple([np.asarray(x) for x in [topic1, t_topic1, topic2, t_topic2]])

    # Search for time of ahead topic which is closest to start of behind topic
    diff = np.abs(t_topic_ahead - t_topic_behind[0])
    # Index where time difference is the smallest
    idx = diff.argmin()
    # Remove first couple values to make both start at the same time
    topic_ahead = topic_ahead[idx:]
    t_topic_ahead = t_topic_ahead[idx:]

    if is_topic1_behind:
        print("topic1 was behind by {}".format(idx))
        return topic_behind, t_topic_behind, topic_ahead, t_topic_ahead
    else:
        print("topic2 was behind by {}".format(idx))
        return topic_ahead, t_topic_ahead, topic_behind, t_topic_behind


def cut_pc(pc2D, x1=1280, y1=720, idx=None):
    """Cuts points of 2D pointcloud which lie outside of camera image"""
    if idx is None:
        idx = (
            (pc2D[:, 0] > 0) & (pc2D[:, 0] < x1) & (pc2D[:, 1] > 0) & (pc2D[:, 1] < y1)
        )
        pc_smol = pc2D[idx]
        return pc_smol, idx
    return pc2D[idx]

In [6]:
def reduce_pc(pc, x_range, y_range, z_range):
    """Removes points outside of box"""
    # Filter array
    pc_cut = pc[
        (pc[:, 0] > x_range[0])
        & (pc[:, 0] < x_range[1])
        & (pc[:, 1] > y_range[0])
        & (pc[:, 1] < y_range[1])
        & (pc[:, 2] > z_range[0])
        & (pc[:, 2] < z_range[1])
    ]
    return pc_cut


def reduce_pc2d(pc, x_range, y_range):
    """Removes points outside of box"""
    # Indices of points inside box
    inliers = [
        (pc[:, 0] > x_range[0])
        & (pc[:, 0] < x_range[1])
        & (pc[:, 1] > y_range[0])
        & (pc[:, 1] < y_range[1])
    ][0]
    # Filter array
    pc_cut = pc[inliers]
    return pc_cut, inliers


def voxel_filter(pc, leaf_size):
    """Downsample point cloud using voxel filter"""
    vgf = pc.make_voxel_grid_filter()
    # Leaf_size is the length of the side of the voxel cube in m
    vgf.set_leaf_size(leaf_size, leaf_size, leaf_size)
    pc_filtered = vgf.filter()
    return pc_filtered


def array_to_pcl(pc_array):
    """Gets pcl point cloud from numpy array"""
    pc = pcl.PointCloud()
    pc.from_array(pc_array.astype("float32"))
    return pc


def ransac(pc):
    """Find inliers and normal vector of dominant plane"""
    # 50?
    seg = pc.make_segmenter_normals(50)
    # Doubles the speed if True
    seg.set_optimize_coefficients(True)
    seg.set_model_type(pcl.SACMODEL_NORMAL_PLANE)
    seg.set_method_type(pcl.SAC_RANSAC)
    # How close a point must be to model to be considered inlier
    seg.set_distance_threshold(0.11)
    # normal_distance_weight?
    seg.set_normal_distance_weight(0.01)
    # How many tries
    seg.set_max_iterations(100)
    inliers_idx, coefficients = seg.segment()
    return inliers_idx, coefficients


def project_pointcloud(pc):
    # Transformation from pointcloud (camera left) to left camera lens (optical) (using output from tf node)
    trans = [0.05999999999999983, -0.3898697311173096, -0.33652837214493014]
    quat = [-0.4473902184573541, 0.4473902184573541, -0.5475782979891378, -0.5475782979891378]

    # Pointcloud points
    objectPoints = np.asarray(pc, dtype=np.float64)

    # Get rotation vector from lidar frame to camera frame
    # Convert quaternion to rotation matrix
    rotMat = quaternion_matrix(quat)[:3, :3]
    # Convert rotation matrix to rotation vector
    rvec, _ = cv2.Rodrigues(rotMat)

    # Translation vector from lidar to camera frame
    tvec = np.asarray(trans)

    # Camera matrix (K)
    cameraMatrix = np.array(
        [
            [519.9877319335938, 0.0, 630.9716796875],
            [0.0, 519.9877319335938, 352.2940673828125],
            [0.0, 0.0, 1.0],
        ]
    )
    cameraMatrix = np.reshape(cameraMatrix, (3, 3))

    # Distortion coefficients (D)
    distCoeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

    # Projection of 3D points onto 2D plane
    lidar_img_points, _ = cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
    lidar_img_points = lidar_img_points.reshape(len(lidar_img_points), 2)
    return lidar_img_points


def mask_lidar_points(pc2d, seg_mask, n=3):
    pc2d_lst = pc2d.astype(int).tolist()

    # Step 1: Get list of pixels of projected points in bounding box
    proj_px = np.zeros(seg_mask.shape)
    # Get coordinates of all projected points as int
    for point in pc2d_lst:
        x, y = point
        proj_px[y - n : y + n, x - n : x + n] = 1

    # Intersect with segmentation mask
    proj_px_mask = np.logical_and(seg_mask, proj_px)

    # Filter out points outside of mask
    # Convert array to list
    proj_px_mask_lst = []
    for x, row in enumerate(proj_px_mask):
        for y, pixel in enumerate(row):
            if pixel == 1:
                proj_px_mask_lst.append([y, x])

    inlier_indices = []
    for point in pc2d_lst:
        if point in proj_px_mask_lst:
            inlier_indices.append(True)
        else:
            inlier_indices.append(False)

    return inlier_indices


def create_plane(coefficients, x_bounds, y_bounds, resolution=0.1):
    """Create plane from coefficients"""
    # Define vectors for grid creation
    xxx = np.arange(x_bounds[0], x_bounds[1], resolution)
    yyy = np.arange(y_bounds[0], y_bounds[1], resolution)
    # Create a grid of points
    xx, yy = np.meshgrid(xxx, yyy)

    # Calculate corresponding z values
    z = -(coefficients[0] * xx + coefficients[1] * yy + coefficients[3]) / coefficients[2]

    # Create pointcloud
    plane = []
    for x_idx, x in enumerate(xxx):
        for y_idx, y in enumerate(yyy):
            plane.append([x, y, z[y_idx, x_idx]])

    plane_arr = np.asarray(plane)
    return plane_arr


def get_plane(pc_arr):
    # Get ramp plane using RANSAC
    pc = array_to_pcl(pc_arr)
    plane_indices, coefficients = ransac(pc)
    pc_plane_inliers_arr = np.asarray(pc.extract(plane_indices))
    normal_vector = coefficients[:3]

    # Create a plane with the coefficients from ransac
    # Limit x and y to the bounds of the point cloud of the ramp region
    xxx_bounds = [pc_plane_inliers_arr[:, 0].min(), pc_plane_inliers_arr[:, 0].max()]
    yyy_bounds = [pc_plane_inliers_arr[:, 1].min(), pc_plane_inliers_arr[:, 1].max()]
    pc_ramp_plane_arr = create_plane(coefficients, xxx_bounds, yyy_bounds)
    return pc_ramp_plane_arr, normal_vector


def angle_calc(v1, v2, degrees=True):
    """Calculate angle between two vectors (planes)"""
    # Assuming both vectors can be rotated alongside one axis to be aligned
    dot = np.dot(v1, v2)

    # Make sure arccos is defined (dot=[0,1]) (should always be the case because
    # v1 and v2 are unit vectors, but probably due to rounding errors not always true)
    if dot <= 1:
        angle = np.arccos(dot)
    else:
        angle = 0

    if degrees is True:
        return np.degrees(angle)
    return angle


Load data (camera image and pointcloud) and extract one frame

In [9]:
# ZED camera image
image, t_image = unpack_bag(bag_path, camera_topic)
# Velodyne point cloud
lidar, t_lidar = unpack_bag(bag_path, lidar_topic)

# Synchronize both topics (zed camera has 30 fps whereas lidar is only 10 Hz)
image, lidar = synchronize_topics(image, t_image, lidar, t_lidar)

# Extract one frame
img = image[frame]
pc = lidar[frame]

# Convert from image msg to numpy array
bridge = CvBridge()
img = bridge.imgmsg_to_cv2(img, desired_encoding="rgb8")
# Convert point cloud msg to numpy array
pc_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(pc, remove_nans=True)

t_start_diff: 0.006 s, t_end_diff: 0.007 s


In [23]:
# Lidar points
# Remove points behind car (because not visible in camera frame)
pc_array = pc_array[pc_array[:, 0] > 0]

# Projection of 3D points onto 2D plane
pc_2d = project_pointcloud(pc_array)
# Remove points which do not lie inside image plane
pc_2d_cut, indices = cut_pc(pc_2d, img.shape[1], img.shape[0])

# Box dimensions
box = np.load("/home/user/objectDetection/predictions_poly/pred_box_lidar_proj.npy")
x, y, w, h = box
# Get box coorinates
x_range = [x, x + w]
y_range = [y, y + h]

# Segmentation mask
seg_mask = np.load("/home/user/objectDetection/predictions_poly/pred_mask_lidar_proj.npy")

# Show image
fig = px.imshow(img)
# Show point cloud
fig.add_trace(
    go.Scatter(
        x=pc_2d_cut[:, 0],
        y=pc_2d_cut[:, 1],
        mode="markers",
        marker=dict(size=2),
        name="Full point cloud",
    )
)
# Visualize bounding box in image
fig.add_shape(
    type="rect",
    x0=x,
    y0=y,
    x1=x + w,
    y1=y + h,
    line=dict(color="blue", width=2),
)
# Crop to bounding box in the point cloud
# Filter points outside of bounding box
pc_ramp_2d_box, inliers_box = reduce_pc2d(pc_2d, x_range, y_range)
fig.add_trace(
    go.Scatter(
        x=pc_ramp_2d_box[:, 0],
        y=pc_ramp_2d_box[:, 1],
        mode="markers",
        marker=dict(size=2),
        name="Bounding box",
    )
)
# Filter points outside of the segmentation mask
inliers_mask = mask_lidar_points(pc_ramp_2d_box, seg_mask)
pc_ramp_2d_mask = pc_ramp_2d_box[inliers_mask]
fig.add_trace(
    go.Scatter(
        x=pc_ramp_2d_mask[:, 0],
        y=pc_ramp_2d_mask[:, 1],
        mode="markers",
        marker=dict(size=2),
        name="Segmentation mask",
    )
)
fig.show()


### Load the detected bounding box from detectron2

Using the trained network, detectron2 outputs a box and mask of the predicted ramp region

This box can now be cut from the image to get the ramp region. Same for the segmentation mask

In [ ]:
# Box dimensions
box = np.load("bounding_box.npy")
x, y, w, h = box
# Get box coorinates
x_range = [x, x + w]
y_range = [y, y + h]

# Segmentation mask
seg_mask = np.load("pred_mask.npy")

# Visualize bounding box in image
fig = px.imshow(img)
fig.add_shape(
    type="rect",
    x0=x,
    y0=y,
    x1=x + w,
    y1=y + h,
    line=dict(color="blue", width=2),
)

# Crop to bounding box in the point cloud
# Filter points outside of bounding box
pc_ramp_2d_box, inliers_box = reduce_pc2d(pc_2d, x_range, y_range)
fig.add_trace(
    go.Scatter(
        x=pc_ramp_2d_box[:, 0],
        y=pc_ramp_2d_box[:, 1],
        mode="markers",
        marker=dict(size=2),
        name="Bounding box",
    )
)
# Filter points outside of the segmentation mask
inliers_mask = mask_lidar_points(pc_ramp_2d_box, seg_mask)
pc_ramp_2d_mask = pc_ramp_2d_box[inliers_mask]
fig.add_trace(
    go.Scatter(
        x=pc_ramp_2d_mask[:, 0],
        y=pc_ramp_2d_mask[:, 1],
        mode="markers",
        marker=dict(size=2),
        name="Segmentation mask",
    )
)
fig.show()

# # Crop to bounding box in the 2D image
# # 2D image
# crop_img = img.copy()
# crop_img = crop_img[box[1] : box[1] + box[3], box[0] : box[0] + box[2]]
# px.imshow(crop_img).show()

# # Crop to bounding box in the depth map
# crop_depth_img = depth_img.copy()
# crop_depth_img = crop_depth_img[box[1] : box[1] + box[3], box[0] : box[0] + box[2]]
# px.imshow(crop_depth_img).show()


In [ ]:
# Extract ramp region from point cloud
pc_ramp_box_arr = pc_small_arr[inliers_box]
pc_ramp_mask_arr = pc_ramp_box_arr[inliers_mask]

# Fit plane inside ramp region
pc_ramp_plane_box_arr, nv_box = get_plane(pc_ramp_box_arr)
pc_ramp_plane_mask_arr, nv_mask = get_plane(pc_ramp_mask_arr)

# Visualize pointcloud with ramp region
fig = go.Figure()
fig.add_trace(
    go.Scatter3d(
        x=pc_small_arr[:, 0],
        y=pc_small_arr[:, 1],
        z=pc_small_arr[:, 2],
        mode="markers",
        marker=dict(size=1),
        name="Full point cloud",
    )
)
fig.add_trace(
    go.Scatter3d(
        x=pc_ramp_box_arr[:, 0],
        y=pc_ramp_box_arr[:, 1],
        z=pc_ramp_box_arr[:, 2],
        mode="markers",
        marker=dict(size=1),
        name="Bounding box inliers",
    )
)
fig.add_trace(
    go.Scatter3d(
        x=pc_ramp_mask_arr[:, 0],
        y=pc_ramp_mask_arr[:, 1],
        z=pc_ramp_mask_arr[:, 2],
        mode="markers",
        marker=dict(size=1),
        name="Segmentation mask inliers",
    )
)
fig.add_trace(
    go.Scatter3d(
        x=pc_ramp_plane_box_arr[:, 0],
        y=pc_ramp_plane_box_arr[:, 1],
        z=pc_ramp_plane_box_arr[:, 2],
        mode="markers",
        marker=dict(size=3),
        name="Bounding box fitted plane",
    )
)
fig.add_trace(
    go.Scatter3d(
        x=pc_ramp_plane_mask_arr[:, 0],
        y=pc_ramp_plane_mask_arr[:, 1],
        z=pc_ramp_plane_mask_arr[:, 2],
        mode="markers",
        marker=dict(size=3),
        name="Segmentation mask fitted plane",
    )
)
fig.show()


### Calcuate ramp properties

Transformation to world frame necessary to get meaningful angle

In [ ]:
# Transformation angles from tf_zed.launch (...  0 -0.174533 0 /base_link /zed2i_camera_center)
tf_angles = [0, -0.174533, 0]
# Transform from camera frame to base_link frame
rot_mat = quaternion_matrix(quaternion_from_euler(*tf_angles))[:3, :3]
pc_ramp_plane_box_arr_rot = np.inner(pc_ramp_plane_box_arr, rot_mat)
pc_ramp_plane_mask_arr_rot = np.inner(pc_ramp_plane_mask_arr, rot_mat)

_, nv_box = get_plane(pc_ramp_plane_box_arr_rot)
_, nv_mask = get_plane(pc_ramp_plane_mask_arr_rot)

def get_ramp_props(ramp_plane_arr, nv):
    angle = angle_calc(nv, (0, 0, 1))
    width = ramp_plane_arr[:, 1].max() - ramp_plane_arr[:, 1].min()
    length = ramp_plane_arr[:, 0].max() - ramp_plane_arr[:, 0].min()
    return angle, width, length

a, w, l = get_ramp_props(pc_ramp_plane_box_arr_rot, nv_box)
print("Bounding box: angle: {:.2f}, width: {:.2f}, length: {:.2f}".format(a, w, l))
a, w, l = get_ramp_props(pc_ramp_plane_mask_arr_rot, nv_mask)
print("Segmentation mask: angle: {:.2f}, width: {:.2f}, length: {:.2f}".format(a, w, l))

# Problem

In [ ]:
# Pointcloud size
n = 10000
# Create random point cloud
pc = np.random.rand(n,3) * 500

def reduce_pc2d(pc, x_range, y_range):
    """Removes points outside of box"""
    # Filter array
    pc_cut = pc[
        (pc[:, 0] > x_range[0])
        & (pc[:, 0] < x_range[1])
        & (pc[:, 1] > y_range[0])
        & (pc[:, 1] < y_range[1])
    ]
    return pc_cut

# Box dimensions (x0, y0, width, height)
box = [437, 399, 240, 121]
# Get box coordinates
x_range = [box[0], box[0] + box[2]]
y_range = [box[1], box[1] + box[3]]

# Crop to bounding box in the point cloud
pc_box = reduce_pc2d(pc_ramp, x_range, y_range)

In [ ]:
# STEP 1: Get region using poly points (I gussed my self and did not use predicition)
# Because prediction gives true, false for every pixel and projected points are not at every pixel
pts = np.array([[446, 525], [664, 526], [607, 405], [522, 405]])
# pts = np.array([[446, 525], [664, 526], [522, 405]])

# STEP 2: Crop the bounding rect
box = [437, 399, 240, 121]
x, y, w, h = box
# rect = cv2.boundingRect(pts)
# x, y, w, h = rect
cropped = img[y:y+h, x:x+w].copy()

# STEP 3: Create mask
pts = pts - pts.min(axis=0)

mask = np.zeros(crop_img.shape[:2], np.uint8)
cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
px.imshow(cropped)

# STEP 4: Do bit-op
dst = cv2.bitwise_and(cropped, cropped, mask=mask)

px.imshow(dst)

In [ ]:
# Convert projected pointcloud array to int
pc_ramp_2d_int = pc_ramp_2d.astype(int)

# Convert to image array
pc_ramp_2d_img = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
for pt in pc_ramp_2d_int:
    y, x = pt
    pc_ramp_2d_img[x, y] = 255
    
px.imshow(pc_ramp_2d_img).show()

pc_ramp_2d_img_masked = pc_ramp_2d_img[y:y+h, x:x+w].copy()
dst = cv2.bitwise_and(pc_ramp_2d_img_masked, pc_ramp_2d_img_masked, mask=mask)
px.imshow(dst).show()

In [ ]:
cv2.bitwise_and(pc_ramp_2d, pc_ramp_2d, mask=mask)